Mistral AI LLM evaluation- Model is mistral-large-latest aka mistral-large-2402

In [1]:
from trulens_eval import TruChain, Feedback, Tru, feedback, Select
from trulens_eval.tru_custom_app import instrument
tru = Tru()
import dotenv
dotenv.load_dotenv()
from pinecone_code import PineconeServerless
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage
import os
from trulens_eval import Feedback, Select
from trulens_eval.feedback import Groundedness
from trulens_eval.feedback.provider.openai import OpenAI as fOpenAI
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity


Package protobuf is installed but has a version conflict:
	(protobuf 4.21.12 (/opt/homebrew/lib/python3.11/site-packages), Requirement.parse('protobuf>=4.23.2'))

This package is optional for trulens_eval so this may not be a problem but if
you need to use the related optional features and find there are errors, you
will need to resolve the conflict:

    ```bash
    pip install 'protobuf>=4.23.2'
    ```

If you are running trulens_eval in a notebook, you may need to restart the
kernel after resolving the conflict. If your distribution is in a bad place
beyond this package, you may need to reinstall trulens_eval so that all of the
dependencies get installed and hopefully corrected:
    
    ```bash
    pip uninstall -y trulens_eval
    pip install trulens_eval
    ```

Package scikit-learn is installed but has a version conflict:
	(scikit-learn 1.3.0 (/opt/homebrew/lib/python3.11/site-packages), Requirement.parse('scikit-learn>=1.3.1'))

This package is optional for trulens_eval so th

🦑 Tru initialized with db url sqlite:///default.sqlite .
🛑 Secret keys may be written to the database. See the `database_redact_keys` option of Tru` to prevent this.


In [3]:
prompt  = "You are a friendly chat model. You will respond to the user's message using only the context provided. The context can be anything from a single message to a whole conversation. You can also ask questions to the user."

In [4]:



def parse_conversations(conversations) -> str:
    data = []
    for cluster_id, cluster_df in conversations.items():
        for i, row in cluster_df.iterrows():
            data.append(
                f"{row['start_time']} - {row['speaker']}: {row['text']}"
            )
        data.append("\n\n")
    data = "\n".join(data)
    return data

In [5]:
class RAG_from_scratch:
    @instrument
    def retrieve(self, query: str, in_filter: list) -> list:
        """
        Retrieve relevant text from vector store.
        """
        obj1 = PineconeServerless()
        response1 = obj1.query_pinecone(query, in_filter)
        conversation1_df = obj1.query_delta_conversations()
        #text_list = list(conversation1_df)
        context = parse_conversations(conversation1_df)
        return context
    
    @instrument
    def generate_completion(self, query: str, context_str) -> str:
        """
        Generate answer from context.
        """
        api_key = os.environ["MISTRAL_API_KEY"]
        model = "mistral-large-latest"
        client = MistralClient(api_key=api_key)

        messages = [
            ChatMessage(role="user", content=f"We have provided context information below. \n"
                f"---------------------\n"
                f"{context_str}"
                f"\n---------------------\n"
                f"Given this information, please answer the question: {query}"
            )
            ]

        # No streaming
        completion = client.chat(
            model=model,
            messages=messages,
        ).choices[0].message.content
        print(completion)
        return completion  

    @instrument
    def query(self, query: str, in_filter:list) -> str:
        context_str = self.retrieve(query, in_filter)
        print("CONTEXT ",context_str)
        completion = self.generate_completion(query, context_str)
        return completion,context_str

rag = RAG_from_scratch()

In [6]:

# Initialize provider class
fopenai = fOpenAI()

grounded = Groundedness(groundedness_provider=fopenai)

# Define a groundedness feedback function
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons, name = "Groundedness")
    .on(Select.RecordCalls.retrieve.rets.collect())
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)

# Question/answer relevance between overall question and answer.
f_qa_relevance = (
    Feedback(fopenai.relevance_with_cot_reasons, name = "Answer Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on_output()
)

# Question/statement relevance between question and each context chunk.
f_context_relevance = (
    Feedback(fopenai.qs_relevance_with_cot_reasons, name = "Context Relevance")
    .on(Select.RecordCalls.retrieve.args.query)
    .on(Select.RecordCalls.retrieve.rets.collect())
    .aggregate(np.mean)
)

✅ In Groundedness, input source will be set to __record__.app.retrieve.rets.collect() .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.app.retrieve.args.query .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input question will be set to __record__.app.retrieve.args.query .
✅ In Context Relevance, input statement will be set to __record__.app.retrieve.rets.collect() .


In [7]:
from trulens_eval import TruCustomApp
tru_rag = TruCustomApp(rag,
    app_id = 'RAG mistral',
    feedbacks = [f_groundedness, f_qa_relevance, f_context_relevance])

In [8]:
# if testing with clustering 
#in_filter = ['a4454589-6c49-4d8d-a6f6-c2b5c99ef229', '7c3c7ac7-94ee-4045-b94a-e9b1695c5b7e']
in_filter = []  #if not using clustering

In [9]:

import pandas as pd 
questions= pd.read_csv("Questions_old.csv")
queries = questions['Question']

ground_truths = questions['Groundtruth']
qna = []

In [10]:
queries=queries.to_list()
ground_truths=ground_truths.to_list()

In [ ]:
for query in queries:
    with tru_rag as recording:
        completion,context = rag.query(query, in_filter=[])
        print(completion)
        qna.append([query, completion, ground_truths[queries.index(query)], context])

last_conversation_no fetched from master json:  20207
conversation length:  3
LENGTH delta_conversation_df:  15
CONTEXT  00:15:35 - spk_6: social media has changed your
00:15:36 - spk_0: life 100%? I dont think I would be the person I am if it wasnt for social media. I first came out on Tik Tok and my parents didnt even know thats how they found out. Actually,
00:15:47 - spk_6: Children under the age of 13 should not be allowed on social media.
00:15:58 - spk_5: Well, um its kind of funny because I talked to a lot of organizations with log off. And the main point that they try to get across is that theyre trying to advocate for kids to get cell phones and technology and social media at an older age. Um, and, you know, years ago I would have said thats really stupid. Thats like the drug talk, like, dont do drugs, dont be on social media. Im not gonna listen to that. But then after years of kind of self reflection, I wish someone would have said you dont need Instagram to have a social  

2024-03-10 10:57:40,398 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 10:57:40,479 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 10:57:40,508 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.Authe

The common recommendation is that children under the age of 13 should not be allowed on social media platforms. However, some people argue that social media can have positive effects for children, especially in terms of helping them connect with others and learn about the world. Ultimately, the decision about when a child is ready for social media depends on their maturity level and ability to use technology responsibly. Education about online safety and appropriate use of technology can also be helpful in preparing children for social media use at an older age.
last_conversation_no fetched from master json:  20207
conversation length:  3
LENGTH delta_conversation_df:  15
CONTEXT  00:09:05 - spk_0: Even if youre not on social media, you know, its gonna do that anyway. Like if youre searching anything on, like if youre searching something up on safari and you go onto your computer onto Chrome, the ads will probably be the same, you know, its just,  its just technology,
 you know.
00:09:

2024-03-10 10:57:42,493 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=2.
2024-03-10 10:57:42,570 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=2.
2024-03-10 10:57:42,615 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.Authe

Several speakers in the conversation shared their experiences with cyberbullying on social media platforms such as Instagram, TikTok, and Musically. They talked about how it can be emotionally distressing and harmful, leading some to feel low self-esteem and even consider suicide. Speakers mentioned that cyberbullying can be particularly challenging because once the harmful comments are posted, they can be seen by a large audience and are permanent. Some speakers noted that they tried to ignore the bullying by not looking at it or blocking the bullies, but it was not always easy to do so. Others mentioned that they had been bullied for their appearance, weight, or race. Overall, the conversation highlighted the negative impacts of cyberbullying on young people and the need to deal with it in a healthy and supportive way.
Several speakers in the conversation shared their experiences with cyberbullying on social media platforms such as Instagram, TikTok, and Musically. They talked about 

...

2024-03-10 10:59:45,929 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 10:59:45,961 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 10:59:46,033 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.Authe

last_conversation_no fetched from master json:  20207
conversation length:  3
LENGTH delta_conversation_df:  15
CONTEXT  00:13:50 - spk_0: OK. We could keep uh we could give that to Thomas to cover that part of the, the flow.
00:13:56 - spk_0: Um
00:13:57 - spk_0: But is the target to also have a mermaid chart created out of that
00:14:01 - spk_0: ex college
00:14:02 - spk_0: flow chart?
00:16:17 - spk_4: Um
00:16:19 - spk_4: Right.
00:16:20 - spk_4: Did you want just to convert this to Mermaid for
00:16:23 - spk_4: the next Tuesday meeting or do you want the,
00:16:25 - spk_4: the whole front end Ds as a Mermaid?
00:16:57 - spk_0: um It reduces a lot of the logic.
00:16:59 - spk_0: It just, it's the way that the files are loaded and, and stuff like that.
00:17:03 - spk_0: Um So it might just be a separate Mermaid chart altogether.
00:17:06 - spk_0: Um
00:17:07 - spk_0: So this is a nice start to have. So



Based on the provided context, it appears that the current front-end mermaid c

2024-03-10 11:00:08,976 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 11:00:09,000 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 11:00:09,008 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.Authe

last_conversation_no fetched from master json:  20207
conversation length:  3
LENGTH delta_conversation_df:  15
CONTEXT  00:07:04 - spk_0: Um
00:07:06 - spk_0: Last week we started discussing about documenting the current implementation of
00:07:09 - spk_0: DS across Git lab
00:07:11 - spk_0: and Patrick just sort of like a status.
00:07:14 - spk_0: You have uh an update here.



00:04:55 - spk_1: We're also working on enabling the web ID E extension marketplace
00:04:59 - spk_1: uh for uh third party extensions within the web ID E.
00:05:03 - spk_1: Uh This will be enabled for GIT lab.com uh
00:05:06 - spk_1: SAS users only in January and we'll be working on
00:05:10 - spk_1: adding it for self manage users uh after that. Uh
00:25:14 - spk_5: to read those responses as they're streaming in.
00:25:16 - spk_5: This will help chat feel more performant and faster at responding to questions.
00:25:21 - spk_5: This will mirror how this works in the git lab
00:25:24 - spk_5: uh dot com U I. 

2024-03-10 11:00:31,639 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 11:00:31,649 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.AuthenticationError'>=Error code: 401 - {'error': {'message': 'Incorrect API key provided: "sk-t9Bc************************************************rtaj. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}. Retries remaining=3.
2024-03-10 11:00:31,667 ERROR trulens_eval.feedback.provider.endpoint.base: openai request failed <class 'openai.Authe

In [ ]:
for q,a,con,gt in qna:
    print(f"Query: {q}\nAnswer: {a}\nContext: {con}", f"\nGround Truth: {gt}\n\n")

Storing results

Mistral AI

In [12]:
import pandas as pd
res = pd.DataFrame(qna, columns = ['Query', 'Answer', 'Ground Truth','Context'])
res.to_csv('rag_results_mistrallarge.csv', index = False)

In [14]:
results_df=tru.get_records_and_feedback(app_ids=["RAG mistral"])[0]
results_df.to_csv("mistrallarge_results.csv")
#results_df = results_df.loc[:,['record_id','input','output','Groundedness','Answer Relevance','Context Relevance','latency','total_tokens','total_cost']].copy()
#results_df.to_csv("mistrallarge_results.csv")
results_df=results_df.tail(11)
#results_df.to_csv("mistrallarge_results.csv")


In [15]:


df=pd.read_csv("rag_results_mistrallarge.csv")
col1=df['Answer']
col2=df['Ground Truth']

def calculate_cosine_similarity(text1, text2):
    vectorizer = CountVectorizer().fit_transform([text1, text2])
    vectors = vectorizer.toarray()
    return cosine_similarity([vectors[0]], [vectors[1]])[0][0]

results_df['cosine_similarity']= df.apply(lambda row: calculate_cosine_similarity(row['Answer'], row['Ground Truth']), axis=1)

results_df.to_csv("mistrallarge_results.csv")